In [19]:
import gradio as gr
import requests
from typing import List, Dict
from config import OPENFDA_API_KEY, OPENFDA_API_URL

In [23]:
class DrugSearch:
    def __init__(self):
        """약물 검색 모듈 초기화"""
        self.api_key = OPENFDA_API_KEY
        
    def search_drugs(self, condition: str) -> List[Dict[str, str]]:
        """질병/증상에 따른 약물 검색"""
        try:
            if not condition.strip():
                return []
                
            url = f"{OPENFDA_API_URL}event.json"
            params = {
                "api_key": self.api_key,
                "search": f"patient.drug.drugindication:{condition} AND patient.reaction.reactionmeddrapt:{condition}",  # 검색 조건 강화
                "count": "patient.drug.medicinalproduct.exact",
                "limit": 10
            }
            
            response = requests.get(url, params=params)
            data = response.json()
            
            if "results" not in data:
                return []
            
            drugs = []
            for result in data["results"]:
                drug_info = {
                    "brand_name": result.get("term", "N/A"),
                    "count": result.get("count", 0),
                    "usage": self.get_drug_purpose(result.get("term", ""), condition),  # 용도 설명 개선
                    "generic_name": "N/A",
                    "manufacturer": "N/A"
                }
                drugs.append(drug_info)
            
            return drugs
            
        except Exception as e:
            print(f"약물 검색 오류: {e}")
            return []
        
    def get_drug_purpose(self, drug_name: str, condition: str) -> str:
        """약물의 주요 용도 설명"""
        condition_purposes = {
            "hypertension": "고혈압 치료",
            "diabetes": "당뇨병 치료",
            "migraine": "편두통 치료",
            "allergy": "알레르기 치료",
            "depression": "우울증 치료"
        }
    
        return condition_purposes.get(condition.lower(), f"{condition} 관련 치료")    
    
    def format_drug_info(self, drugs: List[Dict[str, str]]) -> str:
        """약물 정보를 보기 좋게 포맷팅"""
        if not drugs:
            return "검색 결과가 없습니다."
            
        formatted_text = "# 검색된 약물 목록\n\n"
        formatted_text += "※ 처방 횟수는 FDA에 보고된 데이터 기준입니다.\n"
        formatted_text += "※ 실제 처방 시에는 반드시 의사와 상담하세요.\n\n"
        
        for i, drug in enumerate(drugs, 1):
            formatted_text += f"## {i}. {drug['brand_name']}\n\n"
            formatted_text += f"🔹 주요 용도: {drug['usage']}\n"
            formatted_text += f"🔹 보고된 처방 건수: {drug['count']:,}건\n"
            formatted_text += "-" * 50 + "\n\n"
        
        formatted_text += "\n> 참고: 이 정보는 FDA 데이터를 기반으로 하며, 실제 진료 시에는 의사의 처방을 따라야 합니다."
        
        return formatted_text    
    
    def create_interface(self) -> gr.Blocks:
        """Gradio 인터페이스 생성"""
        with gr.Blocks() as interface:
            gr.Markdown("""
                ## 질병/증상별 약물 검색
                질병이나 증상을 입력하면 관련된 약물 정보를 검색합니다.
                
                예시 검색어:
                - hypertension
                - diabetes
                - migraine
                - allergy
                - depression
            """)
            
            with gr.Row():
                condition_input = gr.Textbox(
                    label="질병/증상 입력",
                    placeholder="영문으로 질병이나 증상을 입력하세요 (예: hypertension)",
                    scale=2
                )
                search_button = gr.Button(
                    "약물 검색",
                    scale=1
                )
            
            result_output = gr.Markdown(label="검색 결과")
            
            def search_and_format(condition):
                drugs = self.search_drugs(condition)
                return self.format_drug_info(drugs)
            
            search_button.click(
                fn=search_and_format,
                inputs=condition_input,
                outputs=result_output
            )
            
        return interface  # 여기에 return 문을 추가

In [24]:
if __name__ == "__main__":
    DSearch = DrugSearch()
    demo = DSearch.create_interface()
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [25]:
demo.close()

Closing server running on port: 7860
